In [7]:
import networkx as nx
import matplotlib.pyplot as plt
from numpy import random
import gerrychain   
from gerrychain import Graph, Partition, proposals, updaters, constraints, accept, MarkovChain
from gerrychain.updaters import cut_edges, Tally
from gerrychain.tree import recursive_tree_part
from gerrychain.proposals import recom
from gerrychain.accept import always_accept
from functools import partial

In [8]:
from gerrychain.random import random
random.seed(12345678)

In [3]:
la_graph = Graph.from_file("./la_election/la_election_shapefile.shp")

In [9]:
tot_pop = sum([la_graph.nodes()[v]['TOTPOP'] for v in la_graph.nodes()])

In [10]:
tot_pop

4657757

In [11]:
num_dist = 6
ideal_pop = tot_pop/num_dist
initial_plan = recursive_tree_part(la_graph, range(num_dist), ideal_pop, 'TOTPOP', 0.02, 10)

In [12]:
initial_partition = Partition(
    la_graph,
    assignment=initial_plan,
    updaters={
        "our cut edges": cut_edges,
        "district population": Tally("TOTPOP", alias="district population")
    }
)

In [13]:
initial_partition

<Partition [6 parts]>

In [15]:
initial_partition["our cut edges"]

{(5, 2877),
 (8, 13),
 (8, 17),
 (10, 11),
 (10, 13),
 (10, 23),
 (10, 2920),
 (11, 15),
 (12, 13),
 (15, 2874),
 (15, 2877),
 (16, 19),
 (17, 41),
 (17, 42),
 (17, 52),
 (17, 54),
 (18, 19),
 (18, 39),
 (18, 41),
 (19, 34),
 (19, 1551),
 (49, 52),
 (49, 54),
 (85, 100),
 (86, 88),
 (86, 94),
 (86, 100),
 (87, 99),
 (87, 100),
 (87, 101),
 (93, 94),
 (93, 135),
 (94, 129),
 (96, 99),
 (96, 119),
 (96, 123),
 (96, 128),
 (96, 136),
 (97, 1853),
 (97, 1872),
 (97, 1873),
 (97, 1883),
 (98, 99),
 (118, 1852),
 (118, 1853),
 (118, 2773),
 (118, 2782),
 (118, 2814),
 (119, 130),
 (130, 132),
 (131, 137),
 (132, 137),
 (135, 1872),
 (135, 1896),
 (136, 137),
 (139, 2782),
 (512, 549),
 (538, 552),
 (540, 545),
 (540, 552),
 (542, 616),
 (543, 549),
 (543, 616),
 (548, 550),
 (548, 557),
 (549, 604),
 (552, 554),
 (554, 556),
 (555, 556),
 (555, 571),
 (555, 616),
 (557, 584),
 (557, 652),
 (597, 621),
 (598, 621),
 (604, 622),
 (621, 622),
 (621, 1560),
 (622, 623),
 (631, 640),
 (631, 2417)

In [16]:
rw_proposal = partial(recom, pop_col="TOTPOP", pop_target=ideal_pop, epsilon=0.02, node_repeats=1)

In [17]:
population_constraint = constraints.within_percent_of_ideal_population(
    initial_partition,
    0.02,
    pop_key="district population"
)

In [21]:
random_walk = MarkovChain(
    proposal = rw_proposal,
    constraints = [population_constraint],
    accept = accept.always_accept,
    initial_state = initial_partition,
    total_steps = 500
)